# Feature engineering & selection

In [23]:
import pandas as pd

TRAINING_SET_FILE = 'data/training_set.csv'

df = pd.read_csv(TRAINING_SET_FILE, sep=';')
df.T
df = ori

df['fecha_reserva_weekday'] = df.fecha_reserva.dt.weekday
df['fecha_reserva_day'] = df.fecha_reserva.dt.day

df['checkin_weekday'] = df['check-in'].dt.weekday
df['checkin_month'] = df['check-in'].dt.month
df['checkin_day'] = df['check-in'].dt.day

df['checkout_weekday'] = df['check-out'].dt.weekday
df['checkout_month'] = df['check-out'].dt.month
df['checkout_day'] = df['check-in'].dt.day

ori = df

In [11]:

# convertimos a datetimes
date_columns = ['fecha_reserva', 'check-in', 'check-out']
for d in date_columns:
    df[d] = pd.to_datetime(df[d])

df['reservation_days_ago'] = (df['check-in'] - df['fecha_reserva']).dt.days
df['total_noches'] = (df['check-out'] - df['check-in']).dt.days
df['precio_por_noche'] = df['importe_reserva'] / df['roomnigths']

groupby_country = df.groupby(by='pais_cliente')['is_reserva_cancelada']
mean_country_cancelations = groupby_country.sum() / groupby_country.count()

mean_country_cancelations = pd.DataFrame(mean_country_cancelations)\
    .rename(columns={'is_reserva_cancelada': 'pct_cancelacion_pais'})
mean_country_cancelations.reset_index(level=0, inplace=True)

df = df.merge(mean_country_cancelations, left_on='pais_cliente', right_on='pais_cliente')

#### Porcen Reser vs Cancelacion por pais

In [21]:
res_por_pais = pd.pivot_table(ori,index=["pais_cliente"],values=["index"], aggfunc=lambda x: len(x.unique()))
can_por_pais = pd.pivot_table(ori,index=["pais_cliente"],values=["is_reserva_cancelada"], aggfunc=sum)
result = pd.concat([res_por_pais, can_por_pais], axis=1, join='inner',join_axes=[res_por_pais.index])
result["%"] = result["is_reserva_cancelada"]*100/result["index"]
result = result.sort_values("%", ascending=False)

In [27]:
res_por_fecha_reserva = pd.pivot_table(ori,index=["fecha_reserva"], columnsvalues=["index"], aggfunc=lambda x: len(x.unique()))
can_por_fecha_reserva = pd.pivot_table(ori,index=["fecha_reserva"],values=["is_reserva_cancelada"], aggfunc=sum)
result_fecha_reserva = pd.concat([res_por_fecha_reserva, can_por_fecha_reserva], axis=1, join='inner',join_axes=[res_por_fecha_reserva.index])
result_fecha_reserva["%"] = result_fecha_reserva["is_reserva_cancelada"]*100/result_fecha_reserva["index"]
result_fecha_reserva.sort_values("%", ascending=False)

,index,is_reserva_cancelada,%
fecha_reserva,,,
2017-06-04,2394,813,33.959900
2017-06-05,3762,1198,31.844763
2017-06-01,3505,1115,31.811698
2017-06-03,2419,754,31.169905
2017-06-08,3843,1174,30.549050
2017-06-02,3497,1062,30.368888
2017-06-06,4038,1212,30.014859
2017-06-07,3910,1172,29.974425


# Feature selection

De todas las características que hemos construido, vamos a evaluar como funcionan.

## Date extractions

In [ ]:
pd.get_dummies(df.fecha_reserva)

In [24]:
ori

,index,fecha_reserva,pais_cliente,importe_reserva,roomnigths,check-in,check-out,is_reserva_cancelada,total_habitaciones,hotel,...,total_noches,precio_por_noche,fecha_reserva_weekday,fecha_reserva_day,checkin_weekday,checkin_month,checkin_day,checkout_weekday,checkout_month,checkout_day
0,0,2017-06-01,Costa Rica,0.003693,3,2017-07-21,2017-07-24,1,1,02bdf29aa2f019efff360097aea1c69e48904c9f,...,3,0.001231,3,1,4,7,21,0,7,21
1,1,2017-06-01,España,0.008979,7,2017-06-11,2017-06-18,1,1,d46b813cd813e2318a35509103b1f081db5f77a3,...,7,0.001283,3,1,6,6,11,6,6,11
2,2,2017-06-01,Reino Unido,0.004807,2,2017-06-16,2017-06-18,0,1,5e15fec283ff9ceb8449e558d8c5c81ac9fd2cfc,...,2,0.002403,3,1,4,6,16,6,6,16
3,3,2017-06-01,España,0.001905,2,2017-06-11,2017-06-13,0,1,3c0e030d4ab643f402af566b86266cbaf6a610ef,...,2,0.000953,3,1,6,6,11,1,6,11
4,4,2017-06-01,España,0.004168,4,2017-09-10,2017-09-14,1,1,d46b813cd813e2318a35509103b1f081db5f77a3,...,4,0.001042,3,1,6,9,10,3,9,10
5,5,2017-06-01,República Dominicana,0.012608,6,2017-06-29,2017-07-02,0,2,290d5f265e3662e4df4e4e4a90f1612cc68404f8,...,3,0.002101,3,1,3,6,29,6,7,29
6,6,2017-06-01,España,0.004959,4,2017-09-10,2017-09-14,1,1,28894d8f7f388a274b0bde507e15bb3d4cbbbb3f,...,4,0.001240,3,1,6,9,10,3,9,10
7,7,2017-06-01,España,0.011972,7,2017-06-11,2017-06-18,1,1,d46b813cd813e2318a35509103b1f081db5f77a3,...,7,0.001710,3,1,6,6,11,6,6,11
8,8,2017-06-01,España,0.002015,2,2017-06-02,2017-06-04,0,1,28894d8f7f388a274b0bde507e15bb3d4cbbbb3f,...,2,0.001008,3,1,4,6,2,6,6,2
9,9,2017-06-01,España,0.008979,7,2017-06-11,2017-06-18,1,1,d46b813cd813e2318a35509103b1f081db5f77a3,...,7,0.001283,3,1,6,6,11,6,6,11
